In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from langdetect import detect

In [2]:
# Load the Excel file into a DataFrame
df = pd.read_excel('app_data.xlsx', engine='openpyxl')

# Display the first few rows of the DataFrame to check if it's loaded correctly
print(df.head())

   product_id product_first_release_date  product_name  \
0  1321171682                 2018-01-04           NaN   
1  1376092414                 2018-04-26  !Arrow Match   
2  1355027915                 2018-04-05   !Ball Chase   
3  1370165429                 2018-04-16  !Dice In Cup   
4  1447150646                 2019-01-05    !Fall Ball   

               product_subtitle product_unified_category_name  \
0  Play Bridge with the best AI                         Games   
1             Swipe to Match...                         Games   
2          Just merge the balls                         Games   
3       Find the hidden dice...                         Games   
4    Show off your fall skills!                         Games   

  publisher_name_cleaned  publisher_id  \
0        GREAT GAME PROD        202565   
1           PRITI KALONI        391930   
2         MULTIPLE CODES        339043   
3           PRITI KALONI        391930   
4                 MEMPIC        318172   

       

In [5]:
## Drop duplicates
# Number of rows before removing duplicates
initial_row_count = df.shape[0]

# Remove duplicate rows based on 'product_id'
df = df.drop_duplicates(subset='product_id')

# Number of rows after removing duplicates
after_duplicates_row_count = df.shape[0]

# Calculate how many rows were dropped
duplicates_dropped = initial_row_count - after_duplicates_row_count
print(f'Number of duplicate rows dropped: {duplicates_dropped}')


Number of duplicate rows dropped: 0


In [3]:
# Concatenate 'product_name', 'product_subtitle', and 'product_description'
df['text'] = df['product_name'].fillna('') + ' ' + df['product_subtitle'].fillna('') + ' ' + df['product_description'].fillna('')

# Detect the language of each text entry (if needed)
df['language'] = df['text'].apply(lambda x: detect(x) if pd.notnull(x) else 'unknown')

# Display the detected languages
print(df['language'].value_counts())


language
en       136400
zh-cn     10487
pt         3090
ja         2433
es         2108
ko         2076
da         1909
vi         1208
tr         1204
ar         1101
fr         1091
de          925
sv          886
ru          708
it          463
id          414
nl          272
th          267
pl          190
no          158
sq          150
zh-tw       140
ca          138
cs          123
ro           89
hr           81
he           65
uk           64
hu           55
sk           44
af           36
el           33
fa           30
fi           28
lt           26
tl           22
sl           18
et           18
cy           16
hi           14
lv            9
so            9
bg            9
sw            6
mk            3
ur            2
mr            1
ne            1
Name: count, dtype: int64


In [4]:
# Initialize the stemmer and stopwords
nltk.download('punkt')
nltk.download('stopwords')
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

# Function to clean and stem text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)  # Remove punctuation and special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    words = nltk.word_tokenize(text)
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    return ' '.join(words)

# Focus on English texts
df_english = df[df['language'] == 'en']

# Apply text cleaning
df_english['cleaned_text'] = df_english['text'].apply(clean_text)

[nltk_data] Downloading package punkt to /Users/siweiran/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/siweiran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/var/folders/t6/61zchkjx5g9b4pgj5_m3fcnr0000gn/T/ipykernel_20002/4221747793.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_english['cleaned_text'] = df_english['text'].apply(clean_text)


In [5]:
# Convert 'product_first_release_date' to datetime format
df_english['product_first_release_date'] = pd.to_datetime(df_english['product_first_release_date'], errors='coerce')

# Extract year and month for grouping
df_english['year_month'] = df_english['product_first_release_date'].dt.to_period('M')

# Sort data by release date
df_english = df_english.sort_values(by='product_first_release_date')

# Split the dataset by category
games_df = df_english[df_english['product_unified_category_name'] == 'Games']
education_df = df_english[df_english['product_unified_category_name'] == 'Education']


/var/folders/t6/61zchkjx5g9b4pgj5_m3fcnr0000gn/T/ipykernel_20002/1525826957.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_english['product_first_release_date'] = pd.to_datetime(df_english['product_first_release_date'], errors='coerce')
/var/folders/t6/61zchkjx5g9b4pgj5_m3fcnr0000gn/T/ipykernel_20002/1525826957.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_english['year_month'] = df_english['product_first_release_date'].dt.to_period('M')


# TF-IDF

## 3 months window within each category

In [16]:
# Define a function to compute similarity for a given category DataFrame
def compute_similarity_for_3_month(category_df):
    vectorizer = TfidfVectorizer()
    similarity_results = []

    for current_month in category_df['year_month'].unique():
        current_month_apps = category_df[category_df['year_month'] == current_month]
        
        # Calculate the start of the 3-month window
        three_months_before = current_month - 3  # Subtract 3 months directly
        
        # Select previous apps within the 3-month window
        previous_apps = category_df[(category_df['year_month'] < current_month) &
                                    (category_df['year_month'] >= three_months_before)]
        
        # If there are no apps within the 3-month window, select all previous apps
        if previous_apps.empty:
            previous_apps = category_df[category_df['product_first_release_date'] < current_month.start_time]

        # Handle case when there are no previous apps at all
        if previous_apps.empty:
            for app_id in current_month_apps['product_id']:
                similarity_results.append({
                    'product_id': app_id,
                    'max_similarity': np.nan,
                    'avg_similarity': np.nan,
                    'top_5_similar_apps': []
                })
            continue

        vectorizer.fit(previous_apps['cleaned_text'])
        previous_tfidf = vectorizer.transform(previous_apps['cleaned_text'])
        current_tfidf = vectorizer.transform(current_month_apps['cleaned_text'])
        
        cosine_sim_matrix = cosine_similarity(current_tfidf, previous_tfidf)
        
        for i, app_id in enumerate(current_month_apps['product_id']):
            sim_scores = cosine_sim_matrix[i]
            top_indices = np.argsort(sim_scores)[-5:][::-1]
            top_similarities = sim_scores[top_indices]
            top_apps = previous_apps.iloc[top_indices]['product_id'].tolist()
            
            similarity_results.append({
                'product_id': app_id,
                'max_similarity': top_similarities[0] if top_similarities.size > 0 else np.nan,
                'avg_similarity': np.mean(top_similarities) if top_similarities.size > 0 else np.nan,
                'top_5_similar_apps': top_apps
            })
    
    return pd.DataFrame(similarity_results)

# Compute similarity separately for Games and Education
games_similarity_results = compute_similarity_for_3_month(games_df)
education_similarity_results = compute_similarity_for_3_month(education_df)

# Save the Games similarity results to a CSV file
games_similarity_results.to_csv('games_similarity_3_months_results.csv', index=False)

# Save the Education similarity results to a CSV file
education_similarity_results.to_csv('education_similarity_3_months_results.csv', index=False)

print("Files have been saved successfully.")

Files have been saved successfully.


## 6 months window within each category

In [17]:
# Define a function to compute similarity for a given category DataFrame
def compute_similarity_for_6_month(category_df):
    vectorizer = TfidfVectorizer()
    similarity_results = []

    for current_month in category_df['year_month'].unique():
        current_month_apps = category_df[category_df['year_month'] == current_month]
        
        # Calculate the start of the 6-month window
        six_months_before = current_month - 6  # Subtract 6 months directly
        
        # Select previous apps within the 6-month window
        previous_apps = category_df[(category_df['year_month'] < current_month) &
                                    (category_df['year_month'] >= six_months_before)]
        
        # If there are no apps within the 6-month window, select all previous apps
        if previous_apps.empty:
            previous_apps = category_df[category_df['product_first_release_date'] < current_month.start_time]

        # Handle case when there are no previous apps at all
        if previous_apps.empty:
            for app_id in current_month_apps['product_id']:
                similarity_results.append({
                    'product_id': app_id,
                    'max_similarity': np.nan,
                    'avg_similarity': np.nan,
                    'top_5_similar_apps': []
                })
            continue

        vectorizer.fit(previous_apps['cleaned_text'])
        previous_tfidf = vectorizer.transform(previous_apps['cleaned_text'])
        current_tfidf = vectorizer.transform(current_month_apps['cleaned_text'])
        
        cosine_sim_matrix = cosine_similarity(current_tfidf, previous_tfidf)
        
        for i, app_id in enumerate(current_month_apps['product_id']):
            sim_scores = cosine_sim_matrix[i]
            top_indices = np.argsort(sim_scores)[-5:][::-1]
            top_similarities = sim_scores[top_indices]
            top_apps = previous_apps.iloc[top_indices]['product_id'].tolist()
            
            similarity_results.append({
                'product_id': app_id,
                'max_similarity': top_similarities[0] if top_similarities.size > 0 else np.nan,
                'avg_similarity': np.mean(top_similarities) if top_similarities.size > 0 else np.nan,
                'top_5_similar_apps': top_apps
            })
    
    return pd.DataFrame(similarity_results)

# Compute similarity separately for Games and Education
games_similarity_results = compute_similarity_for_6_month(games_df)
education_similarity_results = compute_similarity_for_6_month(education_df)

# Save the Games similarity results to a CSV file
games_similarity_results.to_csv('games_similarity_6_months_results.csv', index=False)

# Save the Education similarity results to a CSV file
education_similarity_results.to_csv('education_similarity_6_months_results.csv', index=False)

print("Files have been saved successfully.")

Files have been saved successfully.


## 3 months window of all data

In [18]:
# Compute similarity separately for Games and Education
all_similarity_results = compute_similarity_for_3_month(df_english)

# Save the similarity results to a CSV file
all_similarity_results.to_csv('all_similarity_3_months_results.csv', index=False)

print("Files have been saved successfully.")

Files have been saved successfully.


## 6 months window of all data

In [20]:
# Compute similarity separately for Games and Education
all_similarity_results = compute_similarity_for_6_month(df_english)

# Save the similarity results to a CSV file
all_similarity_results.to_csv('all_similarity_6_months_results.csv', index=False)

print("Files have been saved successfully.")

Files have been saved successfully.


# Classify the publisher into "Big" or "Small" and then use TF-IDF

In [7]:
# Count the number of apps per publisher
app_counts = df_english.groupby('publisher_id').size()

# Classify companies as 'big' or 'small' based on the count of published apps
# If the count of apps is greater than 4, it's a big company (1), otherwise small (0)
df_english['company_size'] = df_english['publisher_id'].apply(lambda x: 1 if app_counts[x] > 4 else 0)

# Inspect the result
print(df_english[['publisher_id', 'company_size']].drop_duplicates())

        publisher_id  company_size
1979          538488             0
48937          87226             0
11508         322778             0
410            68928             0
88416          98904             0
...              ...           ...
2180            5507             1
118532        186609             0
57864         227315             0
123673        519490             0
110994        470135             0

[67379 rows x 2 columns]


## 3-months window of different categories'

In [9]:
# Function to compute similarity for a given category DataFrame within a 3-month window
def compute_similarity_for_3_month(category_df):
    vectorizer = TfidfVectorizer()
    similarity_results = []

    # Loop over each month in the data
    for current_month in category_df['year_month'].unique():
        current_month_apps = category_df[category_df['year_month'] == current_month]
        
        # Calculate the start of the 3-month window
        three_months_before = current_month - 3
        
        # Select previous apps within the 3-month window from big companies only
        previous_apps = category_df[(category_df['year_month'] < current_month) &
                                    (category_df['year_month'] >= three_months_before) &
                                    (category_df['company_size'] == 1)]  # Only big companies

        # If no apps in the 3-month window, select all previous apps from big companies
        if previous_apps.empty:
            previous_apps = category_df[(category_df['product_first_release_date'] < current_month.start_time) &
                                        (category_df['company_size'] == 1)]  # Only big companies

        # Handle case when there are no previous apps at all
        if previous_apps.empty:
            for app_id in current_month_apps['product_id']:
                similarity_results.append({
                    'product_id': app_id,
                    'max_similarity': np.nan,
                    'avg_similarity': np.nan,
                    'top_5_similar_apps': []
                })
            continue

        # Apply TF-IDF to the apps in the previous and current month
        vectorizer.fit(previous_apps['cleaned_text'])
        previous_tfidf = vectorizer.transform(previous_apps['cleaned_text'])
        current_tfidf = vectorizer.transform(current_month_apps['cleaned_text'])
        
        # Compute cosine similarity between current and previous apps
        cosine_sim_matrix = cosine_similarity(current_tfidf, previous_tfidf)

        # Collect results for each app in the current month
        for i, app_id in enumerate(current_month_apps['product_id']):
            sim_scores = cosine_sim_matrix[i]
            top_indices = np.argsort(sim_scores)[-5:][::-1]
            top_similarities = sim_scores[top_indices]
            top_apps = previous_apps.iloc[top_indices]['product_id'].tolist()

            similarity_results.append({
                'product_id': app_id,
                'max_similarity': top_similarities[0] if top_similarities.size > 0 else np.nan,
                'avg_similarity': np.mean(top_similarities) if top_similarities.size > 0 else np.nan,
                'top_5_similar_apps': top_apps
            })
    
    return pd.DataFrame(similarity_results)

# Split the dataset by category
games_df = df_english[df_english['product_unified_category_name'] == 'Games']
education_df = df_english[df_english['product_unified_category_name'] == 'Education']


# Compute similarity within a 3-month window for Games and Education, only comparing to big companies
games_similarity_results = compute_similarity_for_3_month(games_df)
education_similarity_results = compute_similarity_for_3_month(education_df)

# Save the results to CSV files
games_similarity_results.to_csv('games_similarity_3_months_big_companies.csv', index=False)
education_similarity_results.to_csv('education_similarity_3_months_big_companies.csv', index=False)

print("Files have been saved successfully.")


Files have been saved successfully.


## All time similarities in different categories

In [23]:
# Function to compute similarity for a given category DataFrame with all previous apps from big companies
def compute_similarity_with_previous_big_companies(category_df):
    vectorizer = TfidfVectorizer()
    similarity_results = []

    # Loop over each month in the data
    for current_month in category_df['year_month'].unique():
        current_month_apps = category_df[category_df['year_month'] == current_month]
        
        # Select all previous apps from big companies before the current month
        previous_apps = category_df[(category_df['year_month'] < current_month) &
                                    (category_df['company_size'] == 1)]  # Only big companies

        # Handle case when there are no previous apps at all
        if previous_apps.empty:
            for app_id in current_month_apps['product_id']:
                similarity_results.append({
                    'product_id': app_id,
                    'max_similarity': np.nan,
                    'avg_similarity': np.nan,
                    'top_5_similar_apps': []
                })
            continue

        # Apply TF-IDF to the apps in the previous and current month
        vectorizer.fit(previous_apps['cleaned_text'])
        previous_tfidf = vectorizer.transform(previous_apps['cleaned_text'])
        current_tfidf = vectorizer.transform(current_month_apps['cleaned_text'])
        
        # Compute cosine similarity between current and previous apps
        cosine_sim_matrix = cosine_similarity(current_tfidf, previous_tfidf)

        # Collect results for each app in the current month
        for i, app_id in enumerate(current_month_apps['product_id']):
            sim_scores = cosine_sim_matrix[i]
            top_indices = np.argsort(sim_scores)[-5:][::-1]
            top_similarities = sim_scores[top_indices]
            top_apps = previous_apps.iloc[top_indices]['product_id'].tolist()

            similarity_results.append({
                'product_id': app_id,
                'max_similarity': top_similarities[0] if top_similarities.size > 0 else np.nan,
                'avg_similarity': np.mean(top_similarities) if top_similarities.size > 0 else np.nan,
                'top_5_similar_apps': top_apps
            })
    
    return pd.DataFrame(similarity_results)

# Compute similarity for Games and Education, comparing with all previous apps from big companies
games_similarity_results = compute_similarity_with_previous_big_companies(games_df)
education_similarity_results = compute_similarity_with_previous_big_companies(education_df)

# Save the results to CSV files
games_similarity_results.to_csv('games_similarity_all_previous_big_companies.csv', index=False)
education_similarity_results.to_csv('education_similarity_all_previous_big_companies.csv', index=False)

print("Files have been saved successfully.")

Files have been saved successfully.


## 3-months window under all data

In [27]:
# Apply the function to the entire dataset (without separating by category)
similarity_results = compute_similarity_for_3_month(df_english)

# Save the results to a CSV file
similarity_results.to_csv('similarity_3_months_big_companies.csv', index=False)

print("File has been saved successfully.")

File has been saved successfully.


## All time similarities under all data

In [26]:
# Apply the function to the entire dataset (without separating by category)
similarity_results = compute_similarity_with_previous_big_companies(df_english)

# Save the results to a CSV file
similarity_results.to_csv('similarity_all_previous_big_companies.csv', index=False)

print("File has been saved successfully.")

File has been saved successfully.
